In [ ]:
import sys, os
module_path = os.path.abspath(os.getcwd()+'/../..')
print(sys.path)
print(module_path)

sys.path.append(module_path)

In [ ]:
from pydantic import Field
from deepeval.dataset import Golden
from deepeval.test_case import LLMTestCase

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from benchmarq.utility import Evaluator
from benchmarq.experiment import Experiment

from dotenv import load_dotenv

load_dotenv()

model = AutoModelForCausalLM.from_pretrained("gpt2")
tokenizer = AutoTokenizer.from_pretrained("gpt2")
generator = pipeline(task="text-generation", model=model, tokenizer=tokenizer)

# noinspection PyPep8Naming
def generate(prompt: str) -> str:
    return generator(prompt)


class Test(Evaluator):
    def __init__(self):
        super().__init__()

    def evaluate_consumption(self, input: Golden):
        generate(input.input)

    def evaluate_test_case(self, input: Golden) -> LLMTestCase:
        output = generate(f"{input.context}: {input.input}")[0]["generated_text"]
        return LLMTestCase(input=input.input, expected_output=input.expected_output, actual_output=output, context=input.context, retrieval_context=input.retrieval_context)

experiment = Experiment(
    subquestion_id="test_1",
    id="case_1",
    subquestion_path="experiments/test/tests.json",
    name="name",
    description="A very long description",
    settings=Test())

a=experiment.run()


In [ ]:
import os
import pandas as pd

a=pd.read_csv("/Users/leenlaptop/Documents/repos/greenai/green.ai/experiments/test/inputs.csv")

print(a)

In [ ]:
experiment.create_subquestion_json()

In [ ]:
from openai import OpenAI
import time
from benchmarq.utility import VLLMServerSingleton

server = VLLMServerSingleton()



server.start_server()

openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

completion = client.completions.create(model="facebook/opt-125m",
                                      prompt="San Francisco is a")

print("Completion result:", completion)

time.sleep(10)  # Replace with actual usage logic
server.stop_server()